<table style="float:left; border:none">
   <tr style="border:none; background-color: #ffffff">
       <td style="border:none">
           <a href="https://praktikum.yandex.ru/">     
           <img 
               src="https://yastatic.net/q/praktikum/v0.116.12-fix3-1580741332/static/apple-touch-icon.png" 
               style="width:50px"
           >
           </a>    
       </td>
       <td style="border:none">
           <h1>Яндекс.Практикум</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2>SQL анализ рекламных компаний<p>
    </h2></div>

## Описание проекта
Вы аналитик в онлайн-маркетплейсе «Заберу». Это настоящий рай для любителей бесполезных сувениров — там можно приобрести товары ручной работы от небольших производителей. «Заберу» проводит рекламные акции длительностью 1 день. Руководитель отдела маркетинга хочет узнать, насколько эффективны такие кампании.

**В каком виде должно быть решено задание?**

 - Опишите цели исследования
 - Исследуйте таблицы (выведите первые строки)
 - Сделайте по одному SQL-запросу для решения каждой из трёх представленных задач
 - Выведите результаты каждого запроса в тетрадке
 - Опишите выводы по каждой из решённых задач
 
**Доступы к базе данных:**
Следуйте инструкции подключения к базе данных

**Примечания:**
Не забывайте про функции, они сильно облегчат жизнь при выполнении запросов
Ответ должен быть получен с использованием только языка запросов SQL.
Pandas допускается только для хранения и вывода результатов запросов.

## Шаги выполнения проекта:<a id="beginning"></a>


 1) [Подключение к базе данных](#step1)
 
 2) [Исследуем данные](#step2)
 
 3) [**Задача 1:**](#step3)
Посчитайте затраты на каждый рекламный источник.

 4) [**Задача 2:**](#step4)
Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. Сделайте выводы

 5) [**Задача 3:**](#step5)
Введём новую метрику эффективности рекламы - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. 

Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. Рассчитайте для каждого дня введённую метрику. Считайте, что целевое действие было произведено тогда, когда сесия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в рассчёт за 14 апреля

 6) [Финальные выводы](#step6)


**Исходные данные:**

**Таблица visits (лог сервера с информацией о посещениях сайта):**

- uid — уникальный идентификатор пользователя

- device — категория устройства пользователя

- startts — дата и время начала сессии

- endts — дата и время окончания сессии

- sourceid — идентификатор рекламного источника, из которого пришёл пользователь

**Таблица orders (информация о заказах):**

- uid — уникальный идентификатор пользователя, который сделал заказ

- buyts — дата и время заказа

- revenue — выручка Заберу с этого заказа

**Таблица advertisment_costs (информация о затратах на маркетинг):**

- sourceid — идентификатор рекламного источника

- dt — дата

- costs — затраты на этот рекламный источник в этот день

### Подключение к базе данных <a id="step1"></a>

In [4]:
# для подключения запросов необходима библиотека psycopg2, её можно установить через !pip - 
#потребуется установка psycopg2-binary

!pip install psycopg2-binary

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [5]:
# импортируем библиотеки необходимые для выполнения проекта и создадим коннецкцию к базе.
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

[Вернуться к оглавлению](#beginning)

### Исследуем данные<a id="step2"></a>

In [99]:
#изуим список таблиц, имеющихся в базе
engine.table_names()

['orders', 'visits', 'advertisment_costs']

In [5]:
#посмотрим датасет orders

pd.io.sql.read_sql(
''' SELECT 
    * 
    FROM 
    orders;
''', con = engine)

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450
...,...,...,...,...
50410,50411,2018-05-31 23:50:00,$4.64,12296626599487328624
50411,50412,2018-05-31 23:50:00,$5.80,11369640365507475976
50412,50413,2018-05-31 23:54:00,$0.30,1786462140797698849
50413,50414,2018-05-31 23:56:00,$3.67,3993697860786194247


In [93]:
#так же стоит проверить типы данных данного датасета, корретны ли они

query1 = ''' 
    SELECT 
    * 
    FROM 
    orders;
'''
orders = pd.io.sql.read_sql(query1, con = engine)
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 4 columns):
id         50415 non-null int64
buyts      50415 non-null datetime64[ns]
revenue    50415 non-null object
uid        50415 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.5+ MB


In [4]:
#посмотрим датасет visits

pd.io.sql.read_sql(
''' SELECT 
    * 
    FROM 
    visits;
''', con = engine)

,id,uid,device,endts,sourceid,startts
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00
1,1,104060357244891740,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00
2,2,7459035603376831527,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00
3,3,16174680259334210214,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00
4,4,9969694820036681168,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00
...,...,...,...,...,...,...
358527,358527,272012551460639309,desktop,2018-03-20 15:25:00,4,2018-03-20 15:11:00
358528,358528,11171727274084046837,touch,2018-05-15 23:09:00,2,2018-05-15 23:08:00
358529,358529,13482259146982443019,desktop,2018-05-06 20:27:00,5,2018-05-06 20:27:00
358530,358530,3266363999948772866,touch,2018-04-03 11:23:00,5,2018-04-03 11:19:00


In [12]:
#так же стоит проверить типы данных данного датасета, корретны ли они

query = ''' SELECT 
    * 
    FROM 
    visits;
'''
visits = pd.io.sql.read_sql(query, con = engine)
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358532 entries, 0 to 358531
Data columns (total 6 columns):
id          358532 non-null int64
uid         358532 non-null object
device      358532 non-null object
endts       358532 non-null datetime64[ns]
sourceid    358532 non-null int64
startts     358532 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 16.4+ MB


In [6]:
#посмотрим датасет advertisment_costs

pd.io.sql.read_sql(
''' SELECT 
    * 
    FROM 
    advertisment_costs;
''', con = engine)

,id,sourceid,dt,costs
0,1,1,2017-06-01,$75.20
1,2,1,2017-06-02,$62.25
2,3,1,2017-06-03,$36.53
3,4,1,2017-06-04,$55.00
4,5,1,2017-06-05,$57.08
...,...,...,...,...
2537,2538,10,2018-05-27,$9.92
2538,2539,10,2018-05-28,$21.26
2539,2540,10,2018-05-29,$11.32
2540,2541,10,2018-05-30,$33.15


In [98]:
#так же стоит проверить типы данных данного датасета, корретны ли они

query2 = ''' 
    SELECT 
    * 
    FROM advertisment_costs;
         '''

advertisment_costs = pd.io.sql.read_sql(query2, con = engine)
advertisment_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 4 columns):
id          2542 non-null int64
sourceid    2542 non-null int64
dt          2542 non-null datetime64[ns]
costs       2542 non-null object
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 79.6+ KB


Таблицы открыты, пропуски в них отсутствуют, типы данных корректны.

[Вернуться к оглавлению](#beginning)

## Задача 1: <a id="step3"></a>
Посчитайте затраты на каждый рекламный источник.

In [7]:
#Посчитаем затраты на каждый рекламный источник

pd.io.sql.read_sql(
''' SELECT sourceid, SUM(costs) AS total_cost_on_source
    FROM 
    advertisment_costs
    GROUP BY sourceid
    ORDER BY sourceid ASC;
''', con = engine)

,sourceid,total_cost_on_source
0,1,"$20,833.27"
1,2,"$42,806.04"
2,3,"$141,321.63"
3,4,"$61,073.60"
4,5,"$51,757.10"
5,9,"$5,517.49"
6,10,"$5,822.49"


**Вывод:** Основные вложения инвестиций в рекламу приходятся на 3 источник (141 321 тыс у.е.), далее идут источники 4 и 5, с затратами (61 073 у.е. и 51 757 у.е. соответственно), наименьшие инвестиции в источники 9 и 10 (5 517 у.е. и 5 822 у.е. соответственно).

[Вернуться к оглавлению](#beginning)

## Задача №2:<a id="step4"></a>
Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. Сделайте выводы.


In [8]:
# посмотрим, сколько юзеров с компьютеров заходили хотя бы раз с источника 3, подготовим запрос для следующей итерации
pd.io.sql.read_sql(
''' SELECT sourceid, device, uid, startts, endts 
    FROM 
    visits
    WHERE 
        device = 'desktop' and uid IN (SELECT uid from visits where sourceid=3);
''', con = engine)

,sourceid,device,uid,startts,endts
0,3,desktop,9969694820036681168,2017-12-27 14:06:00,2017-12-27 14:06:00
1,3,desktop,4499746016005494365,2017-10-23 12:49:00,2017-10-23 12:58:00
2,2,desktop,2655976123871714644,2018-04-05 09:24:00,2018-04-05 09:27:00
3,3,desktop,15857957287537270437,2018-03-12 23:13:00,2018-03-12 23:25:00
4,3,desktop,6600324468951455066,2018-03-15 14:34:00,2018-03-15 14:41:00
...,...,...,...,...,...
117226,3,desktop,6292692062159304168,2018-02-24 22:51:00,2018-02-24 23:28:00
117227,2,desktop,6036428141437482921,2017-06-19 18:16:00,2017-06-19 18:17:00
117228,3,desktop,1321491773734693040,2017-11-01 14:09:00,2017-11-01 14:10:00
117229,5,desktop,13482259146982443019,2018-05-06 20:27:00,2018-05-06 20:27:00


In [9]:
#Посчитаeм суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК 
#(тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, 
#на который было потрачено больше всего денег.

pd.io.sql.read_sql(
'''WITH desktop3visits AS ( SELECT sourceid, device, uid, startts, endts 
                            FROM 
                            visits
                            WHERE 
                            device = 'desktop' and uid IN (SELECT uid from visits where sourceid=3)
                           )
SELECT  SUM(revenue) AS total_revenue, MAX(revenue) AS max_revenue, MIN(revenue) AS min_revenue
FROM
orders
LEFT JOIN desktop3visits ON orders.uid = desktop3visits.uid 
WHERE orders.buyts BETWEEN desktop3visits.startts AND desktop3visits.endts AND orders.revenue::numeric != 0 ;
''', con = engine)                         

,total_revenue,max_revenue,min_revenue
0,"$64,750.23","$1,195.64",$0.01


In [10]:
# Вариант расчета, где мы не учитываем устройство, с которого был сделан заказ

pd.io.sql.read_sql(
'''SELECT  SUM(revenue) AS total_revenue, MAX(revenue) AS max_revenue, MIN(revenue) AS min_revenue
FROM orders
WHERE uid IN (SELECT uid from visits where sourceid=3 AND device = 'desktop') AND orders.revenue::numeric != 0;
''', con = engine)                         

,total_revenue,max_revenue,min_revenue
0,"$106,058.86","$2,633.28",$0.01


**Вывод:**
Общая выручка с клиентов, сделавших свой заказ с ПК и хотя бы раз заходивших на сайт с источника №3 - 64 750 долл.
Максимальный заказ составляет 1 195 долл, минимальный заказ - 1 цент. Суммарная выручка от клиентов хотя бы раз заходивших на сайт с иточника №3  без диффиренциации по девайсу (ПК или touch), с которого был сделан заказ - 106 058 тыс у.е. Можно сделать вывод, что доля выручки от клиентов, сделавших заказ с ПК выше, чем делавших его с девайса touch. 

Тем не менее с touch был сделан более дорогой максимальный заказ, который принес компании 2 633 у.е.
Можно говорить так же о том, что данный канал инвестирования не окупается, поскольку саоальдо выручки и инвестиций отрицателльно (выручка 106 тыс.у.е.заказов от клиентов, заходивших на сайт с данного источника, против инвестиций - 141 тыс у.е.)

[Вернуться к оглавлению](#beginning)

## Задача №3:<a id="step5"></a>
Введём новую метрику эффективности рекламы - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника.

Рассчитайте для каждого дня введённую метрику. Считайте, что целевое действие было произведено тогда, когда сесия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в рассчёт за 14 апреля.

In [13]:
# посмотрим тотал количество заходов с разных разных источников рекламы по дням
pd.io.sql.read_sql(
''' SELECT sourceid AS source_reklama, COUNT(uid) AS uid_count, TO_CHAR(startts, 'YYYY/MM/DD') AS day 
    FROM 
    visits
    GROUP BY sourceid, day
    ORDER BY sourceid ASC
    ;
''', con = engine)

,source_reklama,uid_count,day
0,1,68,2017/06/01
1,1,60,2017/06/02
2,1,39,2017/06/03
3,1,50,2017/06/04
4,1,85,2017/06/05
...,...,...,...
2577,10,10,2018/05/27
2578,10,45,2018/05/28
2579,10,32,2018/05/29
2580,10,69,2018/05/30


In [12]:
# посмотрим количество уникальных клиентов, приходящих с разных источников рекламы по дням


pd.io.sql.read_sql(
''' SELECT sourceid AS source_reklama, COUNT(DISTINCT uid) AS uid_count, CAST(startts AS date)
    FROM visits
    GROUP BY sourceid, CAST(startts AS date)
    ORDER BY sourceid ASC
    ;
''', con = engine)

,source_reklama,uid_count,startts
0,1,59,2017-06-01
1,1,50,2017-06-02
2,1,37,2017-06-03
3,1,44,2017-06-04
4,1,72,2017-06-05
...,...,...,...
2577,10,10,2018-05-27
2578,10,44,2018-05-28
2579,10,31,2018-05-29
2580,10,68,2018-05-30


In [13]:
# выведем метрику сумм затрат на человека из общего количество зашедших на сайт в день
pd.io.sql.read_sql(
'''WITH day_visits AS ( SELECT (sourceid :: integer) AS source_reklama, COUNT(DISTINCT uid) AS uid_count, CAST(startts AS date)
                        FROM visits
                        GROUP BY sourceid, CAST(startts AS date)
                        ORDER BY sourceid ASC
                       )
    SELECT  id, sourceid, dt, costs, uid_count, advertisment_costs.costs/day_visits.uid_count AS cost_per_person
    FROM advertisment_costs
    LEFT JOIN day_visits ON day_visits.startts = advertisment_costs.dt AND day_visits.source_reklama = advertisment_costs.sourceid
    ;
''', con = engine)

,id,sourceid,dt,costs,uid_count,cost_per_person
0,1,1,2017-06-01,$75.20,59.0,$1.27
1,364,2,2017-06-01,$132.56,79.0,$1.67
2,727,3,2017-06-01,$349.38,182.0,$1.91
3,1090,4,2017-06-01,$76.74,138.0,$0.55
4,1453,5,2017-06-01,$73.24,114.0,$0.64
...,...,...,...,...,...,...
2537,1089,3,2018-05-31,$822.30,394.0,$2.08
2538,1452,4,2018-05-31,$492.54,498.0,$0.98
2539,1816,5,2018-05-31,$398.04,235.0,$1.69
2540,2179,9,2018-05-31,$27.37,62.0,$0.44


**Вывод:**
Метрика "cost_per_person" затрат на 1 посетителя в день по каждому из рекламных источнико рассчитана выше. 

In [90]:
#выведем выручку по источникм и дням для дальнейшего анализа эффективности инвестиций
pd.io.sql.read_sql('''SELECT SUM(revenue) as revenue, DATE_TRUNC('day', buyts)::date as date_revenue, sourceid 
                    FROM orders
                    join visits on orders.uid = visits.uid
                     where orders.buyts between visits.startts and visits.endts    
                     group by date_revenue, sourceid 
                     ORDER BY date_revenue ASC;
''', con = engine)

,revenue,date_revenue,sourceid
0,$159.24,2017-06-01,1
1,$16.68,2017-06-01,2
2,$96.20,2017-06-01,3
3,$28.77,2017-06-01,4
4,$35.92,2017-06-01,5
...,...,...,...
2400,$189.17,2018-05-31,4
2401,$316.50,2018-05-31,5
2402,$7.21,2018-05-31,9
2403,$149.45,2018-05-31,10


In [92]:
# выведем метрику разницы затрат и выручки по дням/источникам/утройствам

pd.io.sql.read_sql('''        SELECT new_visits.date_revenue AS date,
                            new_visits.sourceid,
                            SUM(new_visits.revenue) AS total_revenue,
                            SUM(new_advertisment_costs.costs) AS total_costs,
                            SUM(new_visits.revenue) - SUM(new_advertisment_costs.costs) AS markt_metric
                        
                            FROM
                             (SELECT SUM(revenue) as revenue, DATE_TRUNC('day', buyts)::date as date_revenue, sourceid 
                              FROM orders
                              join visits on orders.uid = visits.uid
                              where orders.buyts between visits.startts and visits.endts    
                              group by date_revenue, sourceid 
                              ORDER BY date_revenue ASC) AS new_visits
                                   
                                INNER JOIN (
                              SELECT
                                    sourceid, 
                                    SUM(costs) AS costs,  
                                    advertisment_costs.dt::date AS date_costs 
                              FROM 
                                    advertisment_costs
                              GROUP BY 
                                    sourceid, 
                                    date_costs
                               ) AS new_advertisment_costs
                               
                                 ON new_visits.sourceid = new_advertisment_costs.sourceid 
                                 AND new_visits.date_revenue = new_advertisment_costs.date_costs
                              GROUP BY 
                                    new_visits.sourceid,
                                    new_visits.date_revenue
                              ORDER BY
                                    date, sourceid;'''
                               , con = engine)

,date,sourceid,total_revenue,total_costs,markt_metric
0,2017-06-01,1,$159.24,$75.20,$84.04
1,2017-06-01,2,$16.68,$132.56,-$115.88
2,2017-06-01,3,$96.20,$349.38,-$253.18
3,2017-06-01,4,$28.77,$76.74,-$47.97
4,2017-06-01,5,$35.92,$73.24,-$37.32
...,...,...,...,...,...
2398,2018-05-31,3,$236.75,$822.30,-$585.55
2399,2018-05-31,4,$189.17,$492.54,-$303.37
2400,2018-05-31,5,$316.50,$398.04,-$81.54
2401,2018-05-31,9,$7.21,$27.37,-$20.16


Судя по таблице выше окупаются у нас источники 1 и 10, остальные являются убыточными, так как сальдо выручки и иснвестиций отрицательно. Лучшим образом у нас отрабатывает источник №10. Источник №3, в которого вкладываются максимальные инвестиции на данном этапе не окупается.

In [15]:
# выведем тотал затраты по сгруппированной таблице и сравним их с тотал затратами из изначального 
#датасета затрат advertisement_costs
pd.io.sql.read_sql(
'''WITH day_visits AS ( SELECT (sourceid :: integer) AS source_reklama, COUNT(DISTINCT uid) AS uid_count, CAST(startts AS date)
                        FROM visits
                        GROUP BY sourceid, CAST(startts AS date)
                        ORDER BY sourceid ASC
                       )
    SELECT  SUM(costs)
    FROM advertisment_costs
    LEFT JOIN day_visits ON day_visits.startts = advertisment_costs.dt AND day_visits.source_reklama = advertisment_costs.sourceid
    ;
''', con = engine)

,sum
0,"$329,131.62"


In [16]:
# суммы совпали, значит группировка была осуществлена корректно
pd.io.sql.read_sql(
''' SELECT 
    SUM(costs)
    FROM 
    advertisment_costs;
''', con = engine)

,sum
0,"$329,131.62"


**Вывод:** Судя по выведеной метрике разницы выручки и инвестиций окупаются источники 1 и 10, остальные являются убыточными, так как сальдо выручки и инвестиций отрицательно. Лучшим образом у нас отрабатывает источник №10. Источник №3, в которого вкладываются максимальные инвестиции на данном этапе не окупается.

[Вернуться к оглавлению](#beginning)

## Общие выводы и заключения<a id="step6"></a>


<div class="col-md-3">
  <ul class="nav flex-column nav-pills">
    <li class="active"><a class="nav-link" id="home-tab" data-toggle="pill" href="#output1" role="tab" aria-controls="output1" aria-selected="true">Посчитайте затраты на каждый рекламный источник.</a></li>
    <li><a class="nav-link" id="profile-tab" data-toggle="pill" href="#output2" role="tab" aria-controls="output2" aria-selected="false">Посчитайте выручку от заказов, сделанных с настольных ПК (тип устройства desktop)</a></li>
    <li><a class="nav-link" id="profile-tab" data-toggle="pill" href="#output3" role="tab" aria-controls="output3" aria-selected="false">Ввод новой метрики эффективности рекламы - дневные затраты на рекламу на пользователя по источнику</a></li>
    <li><a class="nav-link" id="contact-tab" data-toggle="pill" href="#output" role="tab" aria-controls="output" aria-selected="false">Финальные выводы</a></li>
   
  </ul>
</div>


<div class="tab-content">
<div id="output1" class="tab-pane fade in active">
    <h3 style="margin-left:270px">Посчитайте затраты на каждый рекламный источник.</h3>
    <p style="margin-left:270px"> Мы имеем 10 рекламных источников. Инвестиции по 3 из них отсутствуют. Возможно, это связано с технической ошибкой или отсутствием переходов с данных рекламных размещений. <br><br> Основные вложения инвестиций в рекламу приходятся на 3 источник (141 321 тыс у.е.), далее идут источники 4 и 5, с затратами (61 073 у.е. и 51 757 у.е. соответственно), наименьшие инвестиции в источники 9 и 10 (5 517 у.е. и 5 822 у.е. соответственно).</p>
  </div>

<div id="output2" class="tab-pane fade">
    <h3 style="margin-left:270px">Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных с настольных ПК (тип устройства desktop) пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег. Сделайте выводы</h3>
    <p style="margin-left:270px">
Общая выручка с клиентов, сделавших свой заказ с ПК и хотя бы раз заходивших на сайт с источника №3 - 64 750 долл. Максимальный заказ составляет 1 195 долл, минимальный заказ - 1 цент. Суммарная выручка от клиентов хотя бы раз заходивших на сайт с иточника №3 без диффиренциации по девайсу (ПК или tou ch), с которого был сделан заказ - 106 058 тыс у.е. Можно сделать вывод, что доля выручки от клиентов, сделавших заказ с ПК выше, чем делавших его с девайса touch. <br><br>Тем не менее с touch был сделан более дорогой максимальный заказ, который принес компании 2 633 у.е. <br><br> Можно говорить так же о том, что данный канал инвестирования не окупается, поскольку саоальдо выручки и инвестиций отрицателльно (выручка 106 тыс.у.е.заказов от клиентов, заходивших на сайт с данного источника, против инвестиций - 141 тыс у.е.)

</p>
  </div>

  <div id="output3" class="tab-pane fade">
      <h3 style="margin-left:270px">Введём новую метрику эффективности рекламны - дневные затраты на рекламу на пользователя по источнику.</h3>
      <p>
    <p style="margin-left:270px">Судя по выведеной метрике разницы выручки и инвестиций "markt_metric" окупаются источники 1 и 10, остальные являются убыточными, так как сальдо выручки и инвестиций отрицательно. Лучшим образом у нас отрабатывает источник №10. Источник №3, в которого вкладываются максимальные инвестиции на данном этапе не окупается.<br><br>Метрика "cost_per_person" отображает отношение рекламных затрат по данному рекламному источнику к количеству посетителей, перешедших на сайт с данного источника.
            </p>
  </div>
  
  <div id="output" class="tab-pane fade">
      <h3 style="margin-left:270px">Финальные выводы</h3>
      <p>
        <p style="margin-left:270px">  
Мы проанализировали стратегию рекламной кампании маркетплейса «Заберу». Она выстраивается на рекламных акциях длительностью <b>1 день</b>. В целом рекламная кампания показала свою неэффективность.<br><br>
Основные вложения инвестиций в рекламу приходятся на <b>3 источник</b> (141 321 тыс у.е.), далее идут источники 4 и 5, с затратами (61 073 у.е. и 51 757 у.е. соответственно), наименьшие инвестиции в источники 9 и 10 (5 517 у.е. и 5 822 у.е. соответственно)> — 5,5 и 5,8 тыс. соответственно. Основная часть рекламных размещений не окупается. <br><br><b>Видимо, лучше всего отрабатывает рекламный источник №10.</b></p>
            </div>